In [1]:
import time

In [2]:
# load all the reviews on the page
def more_reviews(driver):
    
    links = driver.find_elements_by_xpath('//*[contains(@aria-label,"More reviews")]') # has to change to match stucture
#     print('LINKS HERE', links)
    for l in links:
        l.click()
    time.sleep(5)       
#     links = driver.find_elements_by_xpath('//button[@jsaction=\'pane.reviewChart.moreReviews\']')
#     print('LINKS HERE', links)
#     for l in links:
#         l.click()
#     time.sleep(2)


In [3]:
# scroll down when needed
def scroll_page(driver):
    #has to change to match the new structure
    scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    time.sleep(5)

In [4]:
# Expand full review
def expand_reviews(driver):
    #has to change to match the new structure 
    links = driver.find_elements_by_xpath('//button[@class=\'section-expand-review blue-link\']')
    for l in links:
        l.click()
    time.sleep(5)


In [5]:
## get parsed reviews 
def get_reviews(driver): #number of reviews
    
    from bs4 import BeautifulSoup
    from datetime import datetime
    import pytz
    import pandas as pd
#     import time

    #     time.sleep(4)

    N=100
    more_reviews(driver)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    allrevcontent = soup.find_all('div', class_='ODSEW-ShBeI-content')
    # while len(allrevcontent)< N:
    #     scroll_page(driver)
    #     soup = BeautifulSoup(driver.page_source,'html.parser')
    #     allrevcontent = soup.find_all('div', class_='ODSEW-ShBeI-content')

    expand_reviews(driver)

    revids = []
    for rev in allrevcontent:
        revids.append(rev.find('button',class_='ODSEW-ShBeI-JIbuQc-menu ODSEW-ShBeI-JIbuQc-menu-SfQLQb-title')['data-review-id'])

    users = []
    for rev in allrevcontent:
        users.append(rev.find('div',class_='ODSEW-ShBeI-title').text.replace('\n',' '))
    # #     print('\n')

    revs = []
    for rev in allrevcontent:
        revs.append(rev.find('span',class_='ODSEW-ShBeI-text').text.replace('\n',' '))

    # #     print('\n')
    resps = []
    for rev in allrevcontent:
        try:
            resps.append(rev.find('div',class_='ODSEW-ShBeI-text').text.replace('\n',' '))
        except Exception as e:
            resps.append('None') 
    # #     print('\n')
    revdates = []
    retdates = []
    for rev in allrevcontent:
        revdates.append(rev.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text.replace('\n',' '))

        t        = datetime.now()
        timezone = pytz.timezone('Canada/Eastern')
        t        = timezone.localize(t)
        retdates.append(t)

    rsepdates = []
    # retdates = []
    for r,rev in enumerate(allrevcontent):
        if resps[r]!= 'None':
            rsepdates.append(rev.find('span',class_='ODSEW-ShBeI-QClCJf-QYOpke-VdSJob').text.replace('\n',' '))

            t        = datetime.now()
            timezone = pytz.timezone('Canada/Eastern')
            t        = timezone.localize(t)
        else:
            t='None'

        rsepdates.append(t)

    ratings = []
    for rev in allrevcontent:
        ratings.append(rev.find('span',class_='ODSEW-ShBeI-H1e3jb')['aria-label'])

    review_dict = dict()
    review_dict['revid'] = revids
    review_dict['userid'] = users
    review_dict['review-text'] = revs
    review_dict['response-text'] = resps
    review_dict['review-relative-date'] = revdates
    review_dict['review-retrieve-date'] = retdates
    review_dict['rating'] = ratings

    review_df = pd.DataFrame.from_dict(review_dict)

    
    return review_df
